**因 Jupter 对多进程支持不完善，因此分布式实验的代码运行均需要通过脚本执行！！！！！！**

## 1.背景介绍

本实验主要研究 PyTorch 分布式训练框架下的单机多卡（Single Machine Multi-GPU）流水线并行训练方法，重点探索 torch.distributed.pipelining 的使用。

## 2.实验目的
实现基于 torch.distributed.pipelining 的流水线并行训练，并理解其工作原理。


## 3.硬件要求

2张 GPU（4090、V100、A100等）。


## 4.技术原理

### 流水线并行（Pipeline Parallelism）

流水线并行指将模型的不同部分放在不同的GPU上，从而降低单个GPU显存使用量。

例如，如果有模型为以下结构: [embedding -> pos_encoding -> layer.0 -> layer.1 -> layer.2 -> layer.3 -> out_linear]

我们可以将 [embedding -> pos_encoding -> layer.0 -> layer.1] 这部分模型放到 GPU 0 上，将其余部分 ( [layer.2 -> layer.3 -> out_linear] ) 放到 GPU 1 上。

在前向传播的时候，输入数据经过 GPU 0，计算得到 layer.1 的输出，然后此时 GPU 0 与 GPU 1 之间进行一次张量通信，将 layer.1 的输出发送给 GPU 1。 随后 GPU 1 完成其余部分模型的计算

同理，在反向传播的时候，不同 GPU 之间 也会进行梯度的通信。

### micro-batch 优化

朴素的流水线并行效率不高，是因为同一时间只有一张 GPU 在进行计算。为了提高 GPU 的利用效率，micro-batch 优化技术将一个 input batch 分割成多个 micro-batch， 从而提高数据在多块 GPU 上的流转效率。

## 5.实验流程

### 环境配置



In [ ]:
!pip install torch

### 5.1. transformer模型定义

导入所需要的包，并定义了一个基本的 transformer 模型

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.distributed as dist
import torch.nn.functional as F
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DistributedSampler
import math


from torch.distributed.pipelining import pipeline, SplitPoint
from torch.distributed.pipelining import ScheduleGPipe


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        return x + self.pe[:, : x.size(1)]


class MultiHeadSelfAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        assert d_model % num_heads == 0
        self.d_k = d_model // num_heads
        self.num_heads = num_heads

        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.out_linear = nn.Linear(d_model, d_model)

    def forward(self, x, mask=None):
        batch_size = x.shape[0]

        q = (
            self.q_linear(x)
            .view(batch_size, -1, self.num_heads, self.d_k)
            .transpose(1, 2)
        )
        k = (
            self.k_linear(x)
            .view(batch_size, -1, self.num_heads, self.d_k)
            .transpose(1, 2)
        )
        v = (
            self.v_linear(x)
            .view(batch_size, -1, self.num_heads, self.d_k)
            .transpose(1, 2)
        )

        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float("-1e9"))

        attn_weights = F.softmax(scores, dim=-1)
        output = torch.matmul(attn_weights, v)

        output = (
            output.transpose(1, 2)
            .contiguous()
            .view(batch_size, -1, self.num_heads * self.d_k)
        )
        return self.out_linear(output)


class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))


class TransformerBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.attention = MultiHeadSelfAttention(d_model, num_heads)
        self.norm1 = nn.LayerNorm(d_model)
        self.ffn = FeedForward(d_model, d_ff)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        attn_out = self.attention(x, mask)
        x = self.norm1(x + self.dropout(attn_out))
        ffn_out = self.ffn(x)
        x = self.norm2(x + self.dropout(ffn_out))
        return x


class Transformer(nn.Module):
    def __init__(
        self,
        vocab_size,
        d_model=512,
        num_heads=8,
        d_ff=2048,
        num_layers=6,
        max_len=5000,
    ):
        super(Transformer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model, max_len)
        self.layers = nn.ModuleList(
            [TransformerBlock(d_model, num_heads, d_ff) for _ in range(num_layers)]
        )
        self.out_linear = nn.Linear(d_model, vocab_size)

    def forward(self, x, mask=None):
        x = self.embedding(x)
        x = self.pos_encoding(x)
        for layer in self.layers:
            x = layer(x)
        return self.out_linear(x)


### 5.2.数据集定义

该类定义了一个模拟数据集，其中第 i 条数据类似 [i, i, i, i, i, i, i]

在训练的过程中，我们希望模型能够捕捉到这条性质

In [ ]:
class NLPDataset(Dataset):
    def __init__(self, size, length):
        self.data = []
        for i in range(size):
            self.data.append(torch.full((length, ), i))

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

dataset = NLPDataset(12, 10)
for data in dataset:
    print(data)

### 5.3训练核心代码实现

该段训练代码实现了主要的训练逻辑，包括数据集的使用，模型的实例化，pipeline方法和具体的训练步骤

In [17]:
def train(rank, world_size):
    # Transformer模型的基本参数
    VOCAB_SIZE = 100        # 词表大小
    D_MODEL = 12            # 模型的嵌入维度（hidden size）
    NUM_HEADS = 4           # 多头注意力的头数
    D_FF = 24               # 前馈网络维度
    NUM_LAYERS = 2          # Transformer 层数
    MAX_LEN = 100           # 序列最大长度

    # 数据集相关参数
    DATASET_SIZE = 12       # 数据集中样本数量
    DATASET_LENGTH = 10     # 每个样本的序列长度
    BATCH_SIZE = 4          # 每个 batch 的样本数

    NUM_MICROBATCHES = 2    # 用于 GPipe 分割的 microbatch 数

    # 定义损失函数的封装（方便被 GPipe 调用）
    def compute_loss(output, target):
        criterion = nn.CrossEntropyLoss()
        loss = criterion(output.view(-1, VOCAB_SIZE), target.view(-1))
        return loss

    # 构造 NLP 数据集和 DataLoader
    dataset = NLPDataset(size=DATASET_SIZE, length=DATASET_LENGTH)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE)

    # 构造一个输入模板，用于 GPipe pipeline 初始化
    x = torch.zeros((BATCH_SIZE // NUM_MICROBATCHES, DATASET_LENGTH - 1), dtype=torch.long)

    # 构建带有 split_spec 的 GPipe pipeline，将模型划分为多个 stage
    # 流水线并行的关键是怎么划分一个较大的模型，将模型的不同部分放到不同的 GPU 上。pytorch 的 pipeline 方法，提供了一个基于计算图和编译技术的划分方式，从而在处理不同模型的时候，用户不需要手动将模型切分出来，提高了使用的便利性。
    pipe = pipeline(
        module=Transformer(
            vocab_size=VOCAB_SIZE,
            d_model=D_MODEL,
            num_heads=NUM_HEADS,
            d_ff=D_FF,
            num_layers=NUM_LAYERS,
            max_len=MAX_LEN,
        ),
        mb_args=(x,),  # microbatch 的输入形状，用于模型切分
        split_spec={
            "layers.1": SplitPoint.BEGINNING,  # 从 Transformer 第2层开始划分模型
        },
    )

    # 通过环境变量获取当前进程的rank和world size
    rank = int(os.environ["RANK"])
    world_size = int(os.environ["WORLD_SIZE"])

    # 初始化 PyTorch 分布式通信
    dist.init_process_group(backend="nccl", rank=rank, world_size=world_size)

    # 指定当前 rank 使用的 GPU 设备
    device = f"cuda:{rank}"

    # 获取当前 rank 的模型 stage，用于定义 optimizer
    stage_mod = pipe.get_stage_module(rank)
    optimizer = optim.SGD(stage_mod.parameters(), lr=0.01)

    # 构建模型 stage，并设置 device 和通信信息
    stage = pipe.build_stage(rank, device, None)

    # 创建 GPipe 的训练调度器（ScheduleGPipe）
    # GPipe 调度器具体处理了 microbatch 优化的相关逻辑，负责切分，发送，stage 之间的发送，以及反向传播的过程。
    # 不同于数据并行，流水线并行的调度流程更为复杂，形式上不能和 DDP 一样，warp 成一个带有 forward 方法的 model。例如只有最后一个 stage 需要计算 loss，而且反向传播的正向传播的过程中，不同 stage 都需要进行通信。
    # 而朴素的数据并行在前向传播和反向传播的过程，不同进程组是完全独立的，进程编号不会影响结构，因此实现逻辑上简单一些。
    # 用户在使用的时候，需要将损失函数的计算逻辑传递给 Schedular。
    schedule = ScheduleGPipe(stage, NUM_MICROBATCHES, compute_loss)

    # 正式开始训练过程
    for epoch in range(200):
        for batch, data in enumerate(dataloader):
            # 将输入数据切分为 input 和 target
            label = data[:, 1:].to(device)  # 目标序列（标签）
            x = data[:, :-1].to(device)     # 输入序列（模型输入）

            optimizer.zero_grad()           # 梯度清零

            # 对于 rank 0：只负责前向传输，不计算 loss
            if rank == 0:
                schedule.step(x)
            else:
                # 其他 stage（最后一段）负责 loss 计算和反向传播
                losses = []
                output = schedule.step(target=label, losses=losses)
                print(
                    f"Epoch {epoch}, Batch {batch}, Loss: {torch.stack(losses).mean()}"
                )

            # 优化参数
            optimizer.step()

    # 销毁进程组，结束训练
    dist.destroy_process_group()


### 5.4启动训练

In [ ]:
!bash run.sh